# "Australia's Historical Migration 1996-2020"
> "Some interactive infographics and analytics material, helping to navigate through Australia historical migration statistics."

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [analysis, visualization, australia, migration, jupyter, altair, pandas, insight]
- image: images/2021-10-15-AusMig/sm_preview.png
- hide: false
- search_exclude: true

![]({{ site.baseurl }}/images/2021-10-15-AusMig/cover.png)

# Introduction

Here I recreated a series of charts which I have [previously](https://github.com/mohsen-saki/OZiNDEX) plotted using [`matplotlib`](https://matplotlib.org/) to interrogate some data about Australia's historical migration. Back then I was enjoying learning matplotlib while satisfying my curiosity about the community I live in and its cultural diversity.

Anyway, soon I realized that it would not be much handy and insightful if not interactive. To make up for that, I embedded the infographics in a [Flask web application](https://github.com/mohsen-saki/OZiNDEX/tree/master/app) and [deployed](https://ozindex.herokuapp.com/) it on `Heroku` trying to add some level of automation and interactivity to the dashboard.

This time, I have been using [Altair](https://altair-viz.github.io/index.html), an interesting API library that is easy to create interactive dashboards with and embed them in an `html` base block of code.

# The Dataset

The [dataset](https://data.gov.au/data/dataset/historical-migration-statistics) is held and regularly updated by the federal government. It covers the migration statistics from _1945_ to _2020_ (currently). Like most open datasets published by the government, this one is also a messy set of data due to historical changes in country names and borders, as well as revised classifications and standards.

Previously I used `spreadsheets` to wrangle the data before loading it in a python environment for further exploration. This time I kept it fully reproducible by loading raw data itself into the python environment.

To explore the code, you may see them in collapsed elements of this post, open it in `colab`, or head to [this repo](https://github.com/mohsen-saki/OZiNDEX/tree/master/4blog) for creating a local copy.

**Notes**
1. I just used a portion of the data from 1996 to 2020.
2. The cleaned data set might slightly differ from real numbers as I replaced all `<5` with `5`.

In [2]:
#collapse

from pathlib import Path
import re

import pandas as pd
import numpy as np
import altair as alt

import warnings
warnings.filterwarnings('ignore')

EXCEL_PATH = "https://github.com/mohsen-saki/OZiNDEX/raw/master/\
4blog/2019-20-historical-migration-statistics-locked.xlsx"

In [3]:
#collapse

# utility functions required to process data


# Countrys' name discrepancies between two excel sheets
DIFF_32 = [
    "Bosnia-Herzegovina", "Cape Verde", "Cambodia, the Kingdom of", 
    "China, People's Republic of", "Congo", "Democratic Republic of Congo", 
    "Czech Republic", "Egypt, Arab Republic of", "Swaziland", 
    "Germany, Federal Rep. Of", "Hong Kong (SAR of China)", 
    "Ireland", "Korea, Dem Peoples Rep Of", "Korea, Republic of", 
    "Lao Peoples Democratic Rep", "Macau Spec Admin Rgn", 
    "Netherlands", "Fmr Yugo Rep of Macedonia", "South Sudan", "St Lucia", 
    "South Africa, Republic of", "St Kitts-Nevis", "Syria", 
    "Dem Republic Of Timor-Leste", "Yemen", "British Indian Ocean Terr"]

DIFF_33 =[
    "Bosnia and Herzegovina", "Cabo Verde", "Cambodia", "China, Peoples Republic of (excl SARs)", 
    "Congo, Republic of", "Congo, Dem Republic of The", "Czechia", "Egypt", "Eswatini", 
    "Germany, Fed Republic of", "Hong Kong (SAR of the PRC)", "Ireland, Republic of", 
    "Korea, North", "Korea, South", "Lao Peoples Dem Republic", "Macau (SAR of the PRC)", 
    "Netherlands, Kingdom of The", "North Macedonia", "Republic of South Sudan", 
    "Saint Lucia", "South Africa", "St Kitts and Nevis", "Syrian Arab Republic", "Timor-Leste", 
    "Yemen, Republic of", "British Indian Ocean Territories"]



def load_data(path=EXCEL_PATH):
    """
    to load data from specified excel file
    :param path: path to the excel file
    :return: two instances of pandas dataframe
    
    tip1: header={num} to assumed skiprow={num} applied beforeahnd.
    tip2: using two level header to avoind auto-rename for duplicated country names.
    """
    df1 = pd.read_excel(path, sheet_name="3.2", skiprows=6, 
                        header=[0, 1], skipfooter=10)
    df2 = pd.read_excel(path, sheet_name="3.3", skiprows=6, 
                        header=[0, 1], skipfooter=8)
    
    return df1, df2


def get_transposed_df(df):
    """
    to transpose df in a more redable view
    :param df: pandas df instance
    :return: panda df instance
    """
    df_transpose = df.T
    df_transpose.reset_index(inplace=True)
    df_transpose.columns = df_transpose.iloc[1]
    df_transpose.drop([0,1], inplace=True)
    df_transpose.rename(columns={"Unnamed: 1_level_0":"Stream", "Year":"Country"}, inplace=True)
    df_transpose.reset_index(drop=True, inplace=True)
    
    return df_transpose


def get_clean_df(df):
    """
    to clean df ready for exploration
    :param df: pandas df instance
    :return: panda df instance 
    """
    # removing unneccessary rows/columns
    df = df[
        ~df["Country"].str.contains("total", case=False, regex=False) &
        ~df["Stream"].str.contains("total", case=False, regex=False)
    ]
    
    # stripping white space from column names
    names = df.columns.tolist()
    new_names = [n.strip() for n in names]
    df.rename(columns=dict(zip(names, new_names)), inplace=True)
    
    # removing special characters    
    for c in new_names[2:]:
        df[c] = df[c].astype(str)
        df[c] = df[c].str.extract(r'(\d+)')
        df[c] = df[c].astype(int)
    
    return df


def resolve_country_name(df1, df2, diff1=DIFF_32, diff2=DIFF_33):
    """
    to resolve dicrepancies between country names
    :param df1: first dataframe; column names to be replaced
    :param df2: second dataframe
    :param diff1: countries from df1 with different spelling
    :param diff2: countries from df2 with different spelling
    :return: two panda dataframe instances
    """
    df1.replace(dict(zip(diff1, diff2)), inplace=True)
    
    # removing countries missing from each sheet
    missing_from_df1 = [c for c in df2.Country.unique() if c not in df1.Country.unique()]
    missing_from_df2 = [c for c in df1.Country.unique() if c not in df2.Country.unique()]
    
    df1 = df1[~df1["Country"].isin(missing_from_df2)]
    df2 = df2[~df2["Country"].isin(missing_from_df1)]

    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)
    
    return df1, df2


def get_merged_df(df1, df2):
    """
    to merge two dataframes as final source
    :param df1, df2: pandas datagrame instances
    :return: pandas dataframe
    """
    df = pd.merge(df1, df2, how="outer", 
                  on=["Stream", "Country"])
    df.fillna(0, inplace=True)
    for c in df.columns[2:]:
        df[c] = df[c].astype(int)
    
    return df

In [4]:
#collapse

# to load raw data
sheet_32, sheet_33 = load_data()

# transpose dataset and basic cleaning
tp_32 = get_transposed_df(sheet_32)
tp_33 = get_transposed_df(sheet_33)

# wranglr data further
df_32 = get_clean_df(tp_32)
df_33 = get_clean_df(tp_33)

# dealing with country name discrepancies between to excel sheets
df_32, df_33 = resolve_country_name(df_32, df_33)

# merge tow dataset in a single dataframe
df = get_merged_df(df_32, df_33)

# reshape the dataframe in a long-format table easier for analysis
df = df.melt(id_vars=["Stream", "Country"], var_name="Period",
             value_name="People")
df.head()

,Stream,Country,Period,People
0,Family and Child stream,Afghanistan,1996–97,100
1,Family and Child stream,Albania,1996–97,69
2,Family and Child stream,Algeria,1996–97,20
3,Family and Child stream,American Samoa,1996–97,5
4,Family and Child stream,Andorra,1996–97,0


# Migration Timeline

From the timeline, one observation is that despite [economic growth](https://datacommons.org/place/country/AUS?topic=Economics), migration rate has declined from _2015_. Further down you may explore each country of interest which most of them obviously follows the trend. However, there are few countries like _Brazil_ and _Vietnam_ which keep a continuous positive trend.

Another interesting fact is that almost _%70_ of people  come to Australia on a skill visa stream which data suggests that many come here single or without family. It requires external data to investigate further, but I suppose its social effects on the community could be interesting to have a look at.

In [5]:
#collapse

source = df.groupby(["Stream", "Period"]).sum("People").reset_index()

#----------------------------------------------
"""Creating chart elements"""

# base line chart
line = alt.Chart().mark_line().encode(
    x='Period:O',
    y=alt.Y('People:Q', axis=alt.Axis(format="~s")),
    color=alt.Color('Stream', 
                    scale={"range": ["navy", "darkred", "goldenrod"]}), 
    strokeWidth=alt.value(1),
)

# a selection filter element
nearest = alt.selection(type='single', nearest=True, 
                        on='mouseover', fields=['Period'], empty='none')

# base for rule element and selection points on the chart
selector = alt.Chart().mark_point().encode(
    x='Period:O', 
    opacity=alt.value(0)
).add_selection(nearest)

# adding points based on nearest selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

#adding text tips absed on nearest selection
tooltips = line.mark_text(align='right', dx=-5, dy=-10, 
                          fontWeight=700, fontSize=12).encode(
    text=alt.condition(nearest,
                       alt.Text("People:Q", format=",.0f"),
                       alt.value(' '))
)

# ctreate rule element
ruler = alt.Chart().mark_rule(color='silver').encode(
    x='Period:O'
).transform_filter(nearest)

# layer all elements together
alt.layer(
    line, selector, points, tooltips, ruler, data=source
).properties(
    title="Australia Historical Migration 1996-2020 : Number of People by Migration Stream",
    width=600, 
    height=300,
).configure_axis(
    domain=False, 
    gridDash=[2],
    title=None
).configure_view(
    strokeWidth=0,
).configure_title(
    anchor='start'
)

alt.LayerChart(...)

# Goverment Party

Although Liberal and National coalition managed to form the government for the majority of the past three decades,  data indicates that there is not much debate between two major opposition parties upon migration affairs.

In [6]:
#collapse

source = df.groupby(["Period"]).sum("People").reset_index()

federal_gov = pd.DataFrame([
    {
        "start": "1996–97",
        "end": "2006–07",
        "event": "Coalition (Liberal+)"
    },
    {
        "start": "2006–07",
        "end": "2012–13",
        "event": "Labor"
    },
    {
        "start": "2012–13",
        "end": "2019–20",
        "event": "Coalition (Liberal+)"
    }
])

nearest = alt.selection(type='single', nearest=True, 
                         on="mouseover", fields=["Period"], 
                         empty="none")

line = alt.Chart(source).mark_line().encode(
    x=alt.X("Period:O",),
    y=alt.Y("People:Q", axis=alt.Axis(format="~s")),
    color=alt.value("black"),
    strokeWidth=alt.value(1),
)

points = line.mark_point().encode(
    x=alt.X("Period:O"),
    color=alt.value("yellow"),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
    tooltip=[
        {
            "field" : "People",
            "type" : "quantitative",
            "title" : "People",
            "format" : ",.0f"
        },
        {
            "field" : "Period", 
            "type" : "ordinal", 
            "title" : "Period", 
            }
    ]
).add_selection(nearest)

patch = alt.Chart(federal_gov).mark_rect(opacity=0.6).encode(
    x=alt.X("start:O"),
    x2=alt.X2("end:O"),
    color=alt.Color("event:O", scale={"range": ["deepskyblue", "crimson"]})
)

alt.layer(
    patch,
    line,
    points
).properties(
    title="Australia Historical Migration 1996-2020 : Number of People by Gov Party",
    width=600, 
    height=300,
).configure_axis(
    domain=False, 
    gridDash=[2],
    title=None
).configure_view(
    strokeWidth=0,
).configure_title(
    anchor='start'
)

alt.LayerChart(...)

# Countries' Story

Use this chart to explore any country  of interest. 

One interesting phenomenon with migration statistics is that sometimes those numbers reflect the major economic and social events. For example, the _2014-15_ spike of immigrants from Iran is probably caused by a course of some disastrous events accumulated towards _2013_. Like economic and social failure due to _8_ years of government wrongdoings. As well as the risk of heavier sanctions from the USA followed by United Nation disciplinary actions regarding nuclear debates. By which many Iranina felt like things are about to go unleashed.

In [7]:
#collapse

source = df.groupby(['Country', 'Period']).sum(['People']).reset_index()

dropdown = alt.binding_select(
    options=source.Country.unique())

selector = alt.selection_single(
    fields=['Country'], 
    bind=dropdown, 
    name='Select',
    init={'Country' : "Iran"}
)

bars = alt.Chart(source).transform_joinaggregate(
    total_people='sum(People)'
).transform_calculate(
    perc_of_total="datum.People / datum.total_people"
).mark_bar(
    size=15,
).encode(
    x='Period:O',
    y=alt.Y('People:Q', axis=alt.Axis(format="~s")),
    color=alt.value('teal'),
    tooltip=[
        {
            "field" : "People",
            "type" : "quantitative",
            "title" : "People",
            "format" : ",.0f"
        },
        {
            "field" : "perc_of_total", 
            "type" : "quantitative", 
            "title" : "of Total", 
            "format" : ".0%"
            }
    ]
)

average_line = alt.Chart(source).mark_rule(
    color="gold"
).encode(
    y="mean(People):Q"
)

alt.layer(
    bars, average_line
).add_selection(
    selector
).transform_filter(
    selector
).configure_axis(
    title=None,
    domain=False,
    gridDash=[2]
).configure_view(
    strokeWidth=0
).properties(
    title="Australia Historical Migration 1996-2020 : Number of People by Country",
    width=600, 
    height=300
).configure_title(
    anchor='start'
)

alt.LayerChart(...)

# Top-10 Countries

Following are 10 countries with the most number of people migrating to Australia over almost three decades. Considering the fact that these charts cover the last 25 years, I found the UK next to top a bit surprising.
I also suggest checking out Vietnam. It seems they somehow found a way to hack the system. Different from the general trend for other countries, Vietnamese have a much higher rate of family stream migration compared to skill stream.

In [8]:
#collapse

top_countries = df.groupby(["Country"]).sum(["People"]).sort_values(
    by="People", ascending=False)[:10].reset_index()["Country"].tolist()

source = df[lambda x: x.Country.isin(top_countries)].reset_index(drop=True)


selector = alt.selection_single(
    fields=['Country'], 
    init={"Country": "United Kingdom"},
    empty='none'
)

base = alt.Chart(source)

bars = base.mark_bar(
    size=15,
).encode(
    x=alt.X("sum(People):Q", axis=None),
    y=alt.Y("Country:O", sort="-x",),
    color=alt.Color("sum(People):Q", scale=alt.Scale(scheme="blues"),
                    legend=None),
)

text = bars.mark_text(
    align="left",
    baseline="middle",
    dx=3,
).encode(
    text=alt.Text("sum(People):Q", format=".3s"),
    color=alt.value("black"),
)

layered_bars = alt.layer(
    bars, text
).add_selection(
    selector
).properties(
    width=275, 
    height=200,
    title="Top-10 Countries by People",
)

lines = base.mark_line().encode(
    x=alt.X("Period:O"),
    y=alt.Y("People:Q", axis=alt.Axis(format="~s")),
    color=alt.Color('Stream', 
                    scale={"range": ["navy", "darkred", "goldenrod"]},
                    legend=alt.Legend(orient="top-left")
                   ),
)

dynamic_title = alt.Chart(source).mark_text().encode(
    text=alt.Text("Country:O"),
    color=alt.value("silver")
)

layered_lines = alt.layer(
    lines,
    dynamic_title
).transform_filter(
    selector
).properties(
    width=300, 
    height=200,
    title="Migration Stream by Selected Country"
)

alt.hconcat(
    layered_bars,
    layered_lines
).configure_view(
    strokeWidth=0,
).configure_axis(
    title=None,
    domain=False,
    gridDash=[1]
)

alt.HConcatChart(...)

# Compare

And last, you may compare any tow countries' migration history here. Usually neighbouring countries or those with a complicated history of conflict show interesting insights when statistically compared against each other.

In [9]:
#collapse

source = df.groupby(["Country", "Period"]).sum("People").reset_index()

dropdown = alt.binding_select(
    options=source.Country.unique())

selector1 = alt.selection_single(
    fields=['Country'], 
    bind=dropdown, 
    name='Select_1st',
    init={'Country' : "Hong Kong (SAR of the PRC)"}
)

selector2 = alt.selection_single(
    fields=['Country'], 
    bind=dropdown, 
    name='Select_2nd',
    init={'Country' : "Thailand"}
)

base = alt.Chart(width=600, height=350).encode(
    x=alt.X("Period:O", axis=alt.Axis(title=None))
)

area1 = base.mark_area(
    line={'color':'darkred'},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0),
               alt.GradientStop(color='tomato', offset=1)],
        x1=1,
        x2=1,
        y1=1,
        y2=0)
).encode(
    y=alt.Y("People:Q", axis=alt.Axis(
        format="~s", title=None,
        domainColor="darkred",
        tickColor="darkred",
        labelColor="darkred")),
)

c1 = alt.Chart().mark_text(
    angle=270,
    dx=75,
    dy=350
).encode(
    text=alt.Text("Country:O"),
    color=alt.value("darkred"),
)

layer1 = alt.layer(
     area1, c1
 ).add_selection(
    selector1
).transform_filter(
    selector1
)

area2 = base.mark_area(
    line={'color':'darkgreen'},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0),
               alt.GradientStop(color='seagreen', offset=1)],
        x1=1,
        x2=1,
        y1=1,
        y2=0)
).encode(
    y=alt.Y("People:Q", axis=alt.Axis(
        format="~s", title=None,
        domainColor="seagreen",
        tickColor="seagreen",
        labelColor="seagreen")),
)

c2 = alt.Chart().mark_text(
    angle=270,
    dx=75,
    dy=-330
).encode(
    text=alt.Text("Country:O"),
    color=alt.value("darkgreen"),
)

layer2 = alt.layer(
     area2, c2
 ).add_selection(
    selector2
).transform_filter(
    selector2
)

alt.layer(
    layer2,
    layer1,
    data=source
).resolve_scale(
    y="independent"
).configure_view(
    strokeWidth=0,
).properties(
    title="Australia Historical Migration 1996-2020 : People by Country (to Compare)"
)

alt.LayerChart(...)